In [23]:
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix, triu, tril, hstack
import graphrelations as gr
from iterativemethods import *
from aggregation_coarsening import *


In [24]:
testmat = np.zeros((8,8))
is_edge = [(1,5),(2,4), (2,8), (3,5),(4,2), (4,6), (5,1), (5,3), (5,6), (5,7), (6,4), (6,5), (6,8), (7,5), (8,2), (8,6)]
for i,j in is_edge:
    testmat[i-1,j-1] = 1
testmat = coo_matrix(testmat)
ev = gr.EV(testmat)
ee = gr.EE(ev)

In [25]:
# testweights = [0.1,5.0,1.2,3.0,1.75,2.0,4.0,3.0]

In [26]:
weights = edge_modularity_weights(testmat)
max_edges = luby(ee, weights)
P = vert_agg(max_edges, ev)

In [27]:
print(P.toarray())

[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]


In [28]:
AC_1 = P.T @ testmat @ P
print(AC_1.toarray())
#this is where you check if cfactor is reached. for cfactor = 1.5, we can do one more step
# if 8/1.5=5.3333 >= numverts in AC_n, stop.

[[2. 0. 1. 1. 1. 0.]
 [0. 2. 0. 1. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0.]]


In [29]:
lala = coo(AC_1 - AC_1.diagonal())
print(lala.toarray())

[[ 0. -2.  1.  1.  1.  0.]
 [-2.  0.  0.  1.  0.  1.]
 [-1. -2.  0.  0.  0.  0.]
 [-1. -1.  0.  0.  0.  1.]
 [-1. -2.  0.  0.  0.  0.]
 [-2. -1.  0.  1.  0.  0.]]


In [30]:
ev2 = gr.EV(AC_1)
print(ev2.toarray())

[[1 0 1 0 0 0]
 [0 1 0 1 0 0]
 [1 0 0 1 0 0]
 [1 0 0 0 1 0]
 [0 1 0 0 0 1]
 [0 0 0 1 0 1]]


In [31]:
ev2 = gr.EV(AC_1)
ee2 = gr.EE(ev2)
md = mod_mat_inefficient(AC_1)
print(ee2.toarray())
print(md)

[[0 0 1 1 0 0]
 [0 0 1 0 1 1]
 [1 1 0 1 0 1]
 [1 0 1 0 0 0]
 [0 1 0 0 0 1]
 [0 1 1 0 1 0]]
[[ 0.4375 -1.25    0.6875  0.0625  0.6875 -0.625 ]
 [-1.25    1.     -0.25    0.25   -0.25    0.5   ]
 [ 0.6875 -0.25   -0.0625 -0.1875 -0.0625 -0.125 ]
 [ 0.0625  0.25   -0.1875 -0.5625 -0.1875  0.625 ]
 [ 0.6875 -0.25   -0.0625 -0.1875 -0.0625 -0.125 ]
 [-0.625   0.5    -0.125   0.625  -0.125  -0.25  ]]


In [32]:
weights2 = edge_modularity_weights(AC_1)
print(AC_1.data)
print(ee2.toarray())
print(weights2)
max_edges2 = luby(ee2, weights2)
P2 = vert_agg(max_edges2, ev2)
print(P2.toarray())

[1. 1. 1. 2. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1.]
[[0 0 1 1 0 0]
 [0 0 1 0 1 1]
 [1 1 0 1 0 1]
 [1 0 1 0 0 0]
 [0 1 0 0 0 1]
 [0 1 1 0 1 0]]
[0.6875 0.25   0.0625 0.6875 0.5    0.625 ]
[[1 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]


In [33]:
AC_2 = P2.T @AC_1 @P2
print(AC_2.toarray())
#now cfactor is reached, stop and compose the P's

[[4. 0. 1. 1. 0.]
 [0. 2. 1. 0. 1.]
 [1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0.]]


In [34]:
Pc = P @ P2
print(Pc.toarray())
AC_final = Pc.T @ testmat @ Pc
print(AC_final.toarray())

[[1 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]]
[[4. 0. 1. 1. 0.]
 [0. 2. 1. 0. 1.]
 [1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0.]]


In [35]:
q = P_coarse(testmat, 1.5, modularity_weights=True)

In [14]:
print(q[0].toarray())
print(q[1].toarray())


[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
[[4. 0. 1. 1. 0.]
 [0. 2. 1. 0. 1.]
 [1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0.]]


In [15]:
'''
ok I think it works!! except it feels clunky that I remove the diagonal so many times....
I have a hunch that's going to make some issues now when I try to run it with the laplacian..
'''

"\nok I think it works!! except it feels clunky that I remove the diagonal so many times....\nI have a hunch that's going to make some issues now when I try to run it with the laplacian..\n"

In [16]:
LA = Laplacian(testmat)
print(LA.toarray())
mod_la = LA.copy()
mod_la[LA.shape[0]-1, :-1] = 0
mod_la[:-1,LA.shape[0]-1] = 0
mod_la.eliminate_zeros()
print(mod_la.toarray())

[[ 1.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  2.  0. -1.  0.  0.  0. -1.]
 [ 0.  0.  1.  0. -1.  0.  0.  0.]
 [ 0. -1.  0.  2.  0. -1.  0.  0.]
 [-1.  0. -1.  0.  4. -1. -1.  0.]
 [ 0.  0.  0. -1. -1.  3.  0. -1.]
 [ 0.  0.  0.  0. -1.  0.  1.  0.]
 [ 0. -1.  0.  0.  0. -1.  0.  2.]]
[[ 1.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  2.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0. -1.  0.  0.  0.]
 [ 0. -1.  0.  2.  0. -1.  0.  0.]
 [-1.  0. -1.  0.  4. -1. -1.  0.]
 [ 0.  0.  0. -1. -1.  3.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  2.]]


D:\LinearAlgebra\multilevelmethods\venv\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [17]:
w_la = edge_modularity_weights(mod_la)
print(w_la)

[-1. -1. -1. -1. -1. -1.]


In [18]:
P_coarse_LA, A_coarse_LA = P_coarse(mod_la, modularity_weights=True)

In [19]:
print(P_coarse_LA.toarray())
print(A_coarse_LA.toarray())


[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
[[ 3.  0. -1. -1. -1.  0.]
 [ 0.  2.  0. -1.  0.  0.]
 [-1.  0.  1.  0.  0.  0.]
 [-1. -1.  0.  3.  0.  0.]
 [-1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  2.]]


In [20]:
def fun(A, P):
    if A == 0: return A, P
    return fun(A-1, hstack([P,coo_matrix([[5+A],[6+A],[7+A]])]))

row = np.array([0, 1, 2, 0])
col = np.array([0, 1, 1, 0])
data = np.array([1, 2, 4, 8])
p = csr_matrix((data, (row, col)))
p.toarray()


array([[9, 0],
       [0, 2],
       [0, 4]], dtype=int32)

In [21]:

huh = fun(3, p)
p.toarray()
print(huh)


(0, <3x5 sparse matrix of type '<class 'numpy.intc'>'
	with 12 stored elements in COOrdinate format>)


In [22]:
#testing the iterative methods
b = np.zeros(testmat.shape[0])
xtest= np.random.rand(*b.shape)

#below returns approximate x, number of iterations it took to converge, norm of original error, and norm of error at final iteration
x = stationary_it_method(testmat, b, bgs, xtest, 10000)

2.0175089470879035
0.37140047074296156
0.10096573910377021
0.05638314555817811
0.03698380071955114
0.024227798469491194
0.016288512203288863
0.011063644700241784
0.007619087049866865
0.0054034744518349585
0.004033190581207544
0.0032337157790769074
0.0027994038513181846
0.0025782705729092666
0.002470175086041999
0.0024179053444749565
0.002392123498608077
0.0023786999321620313
0.0023710308825879603
0.002366062748383789
0.0023623826310430873
0.0023593313282586938
0.002356596692867822
0.0023540277413059392
0.0023515493867886208
0.002349123003656054
0.0023467279475879207
0.002344352733623383
0.0023419907196725463
0.0023396379284019494
0.0023372919076646115
0.002334951110280059
0.002332614542758782
0.002330281558801198
0.0023279517339340338
0.0023256247873720856
0.002323300532327908
0.0023209788439233683
0.0023186596382547744
0.0023163428586213024
0.0023140284664251785
0.002311716435131201
0.0023094067462531257
0.002307099386675303
0.0023047943468728965
0.0023024916197166992
0.00230019119968